In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')
# Create models
node = np.empty(4, dtype = object)
node[0] = ClassSwitch(model, 'CS', [[0, 1], [1, 0]])
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[2] = Queue(model, 'Queue2', SchedStrategy.PS)
node[3] = Queue(model, 'Delay', SchedStrategy.INF)

# Create job classes
jobclass = np.empty(2, dtype = object)
jobclass[0] = ClosedClass(model, 'Class1', 15, node[3], 0)
jobclass[1] = ClosedClass(model, 'Class2', 5, node[3], 0)

# Set service times
node[1].set_service(jobclass[0], Exp.fit_mean(1.5))  #mean = 1.5
node[1].set_service(jobclass[1], Erlang.fit_mean_and_order(1.5, 2))  #mean = 1.5

node[2].set_service(jobclass[0], Erlang.fit_mean_and_order(1.5, 2))  #mean = 1.5
node[2].set_service(jobclass[1], Exp.fit_mean(1.5))  #mean = 1.5

node[3].set_service(jobclass[0], Exp.fit_mean(1.0))  #mean = 1
node[3].set_service(jobclass[1], Exp.fit_mean(1.0))  #mean = 1

# Add links
model.add_link(node[1], node[0])
model.add_link(node[2], node[0])
model.add_link(node[0], node[3])
model.add_link(node[3], node[1])
model.add_link(node[3], node[2])

# Set routing strategies
node[0].set_routing(jobclass[0], RoutingStrategy.RAND)
node[1].set_routing(jobclass[0], RoutingStrategy.RAND)
node[2].set_routing(jobclass[0], RoutingStrategy.RAND)
node[3].set_routing(jobclass[0], RoutingStrategy.RROBIN)

node[0].set_routing(jobclass[1], RoutingStrategy.RAND)
node[1].set_routing(jobclass[1], RoutingStrategy.RAND)
node[2].set_routing(jobclass[1], RoutingStrategy.RAND)
node[3].set_routing(jobclass[1], RoutingStrategy.WRROBIN, node[1], 1)
node[3].set_routing(jobclass[1], RoutingStrategy.WRROBIN, node[2], 2)

In [3]:
# Solve with JMT and DES (matches MATLAB)
solver = np.array([], dtype = object)
solver = np.append(solver, JMT(model, verbose=True, seed=23000))
solver = np.append(solver, DES(model, verbose=True, seed=23000))

AvgTable = np.empty(len(solver), dtype = object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].get_name()}')
    AvgTable[s] = solver[s].avg_table()


SOLVER: JMT
SolverJMT command: java -cp /home/gcasale/.virtualenvs/python/lib/python3.13/common/JMT.jar jmt.commandline.Jmt sim /tmp/jmt_oqibn4dp/model.jsimg
Station JobClass    QLen   Util   RespT  ResidT   ArvR   Tput
 Queue1   Class1  1.9929 0.5755  4.7997  4.7997 0.3774 0.3774
 Queue1   Class2  0.6083 0.1905  4.6763  4.6763 0.1255 0.1255
 Queue2   Class1  1.7183 0.5821  4.6923  4.6923 0.3805 0.3805
 Queue2   Class2  0.5794 0.1812  4.7154  4.7154 0.1254 0.1254
  Delay   Class1 11.0061 0.7441 15.1901 15.1901 0.7540 0.7540
  Delay   Class2  3.7916 0.2534 15.2007 15.2007 0.2533 0.2533

SOLVER: DES
Station JobClass   QLen   Util   RespT  ResidT   ArvR   Tput
 Queue1   Class1 1.4540 0.3729  5.7127  5.7127 0.2492 0.2492
 Queue1   Class2 1.4454 0.3748  5.7058  5.7058 0.2480 0.2480
 Queue2   Class1 1.4780 0.3780  5.8433  5.8433 0.2485 0.2484
 Queue2   Class2 1.4881 0.3760  5.8519  5.8519 0.2496 0.2496
  Delay   Class1 7.0749 0.5017 14.3289 14.3289 0.4978 0.4977
  Delay   Class2 7.0596 0.49